In [18]:
using System;
using System.Collections.Generic;
using System.Threading;

In [19]:
public interface ICommand
{
    void Execute();
}

In [23]:
public class ServerThread
{
    private Queue<ICommand> commandQueue = new Queue<ICommand>();
    private Thread thread;

    public void AddCommand(ICommand command)
    {
        lock (commandQueue)
        {
            commandQueue.Enqueue(command);
            Monitor.Pulse(commandQueue);
        }
    }

    public ServerThread()
    {
        thread = new Thread(Run);
        thread.Start();
    }

    private void Run()
    {
        while (true)
        {
            ICommand command = null;
            lock (commandQueue)
            {
                while (commandQueue.Count == 0)
                {
                    Monitor.Wait(commandQueue);
                }
                command = commandQueue.Dequeue();
            }

            if (command != null)
            {
                try
                {
                    command.Execute();
                }
                catch (Exception ex)
                {
                    ExceptionHandler.HandleException(ex, command);
                }
            }
        }
    }
}

In [20]:
public class HardStopCommand : ICommand
{
    public void Execute()
    {
        if (Thread.CurrentThread == Program.serverThread)
        {
            Environment.Exit(0);
        }
        else
        {
            throw new InvalidOperationException("HardStop может быть выполнен только в том потоке, который он должен остановить.");
        }
    }
}

In [21]:
public class SoftStopCommand : ICommand
{
    public void Execute()
    {
        if (Thread.CurrentThread == Program.serverThread)
        {
            Environment.Exit(0);
        }
        else
        {
            throw new InvalidOperationException("SoftStop может быть выполнен только в том потоке, который он должен остановить.");
        }
    }
}

In [22]:
public static class ExceptionHandler
{
    public static void HandleException(Exception ex, ICommand command)
    {
        Console.WriteLine($"Исключение {ex.GetType().Name} обрабатывается для получения команды: {command.GetType().Name}");
    }
}

In [24]:
public class Program
{
    public static Thread serverThread;

    public static void Main()
    {
        ServerThread server = new ServerThread();
        serverThread = Thread.CurrentThread;

        server.AddCommand(new HardStopCommand());
        server.AddCommand(new SoftStopCommand());

        Console.ReadLine(); 
    }
}